<a href="https://colab.research.google.com/github/Daniil81704/Text-clusterization/blob/master/Text_clusterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installation-секция
!pip install -U sentence-transformers
!pip install hdbscan

In [ ]:
# import-секция
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import hdbscan

In [ ]:
sentences = ['Здравствуйте, я хочу заказать пиццу',
             'Добрый день, я бы хотел снять у вас одноместный номер на трое суток',
             'Естественная сладость яблока и моркови гармонично сочетается с солоноватостью сыра',
             'Сочные овощи станут основой теста, мука и яйцо помогут скрепить ингредиенты, а простые специи сбалансируют вкус',
             'Мы с семьей каждый год ходим в поход',
             'Отличная выдалась поездка на море в этом году!']

sentences_arr = np.array(sentences)

In [ ]:
# определение необходимых функций
def obtaining_embeddings(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

def get_clustered(labels):
  x = list(labels)
  res = []
  for i in range(0, max(labels)+1):
    lst_1 = np.where(x == np.int32(i))[0]
    for j in lst_1:
      res.append([i, sentences_arr[j], j])
  return res

In [ ]:
tokenizer = BertTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
model = BertModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

In [ ]:
# получение эмбеддингов
embeddings = []
for i in range(len(sentences)):
  embeddings.append(obtaining_embeddings(sentences[i], model, tokenizer))
embeddings_arr = np.array(embeddings)

In [ ]:
# кластеризация предложений
clusterer = hdbscan.HDBSCAN(min_cluster_size = 2, min_samples=2, cluster_selection_epsilon=0.01, alpha=0.5)

# Обучение на данных
clusterer.fit(embeddings_arr)

# Получение меток кластеров
labels = clusterer.labels_

In [ ]:
res = get_clustered(labels)
print(labels)
print(res)